In [1]:
import pandas as pd
import spacy
import category_encoders as ce
from sklearn.ensemble import RandomForestClassifier
from scipy.stats import randint
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.ensemble import GradientBoostingClassifier
import numpy as np
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import CountVectorizer

# Load Spacy large
nlp = spacy.load("en_core_web_lg")

In [42]:
nlp_cols  = ['blurb', 'name']
cat_cols  = ['country', 'spotlight', 'staff_pick', 'currency']
num_cols  = ['converted_pledged_amount', 'backers_count', 'goal']
target    = 'state'
date_cols = ['launched_at', 'deadline']

In [43]:
df_train = pd.read_pickle('df_train.pkl')
df_test  = pd.read_pickle('df_test.pkl')


X_train, y_train = df_train.drop(columns = [target]), df_train[target]
X_test,   y_test =  df_test.drop(columns = [target]),  df_test[target]

In [18]:
X_train

In [44]:
X_train['deadline']

1      2012-04-02 06:59:00
2      2016-02-13 18:34:51
3      2013-03-14 12:18:28
4      2012-04-01 04:59:00
5      2014-11-15 16:25:29
               ...        
3657   2015-06-06 06:59:00
3658   2012-07-26 20:20:13
3659   2019-08-06 18:00:19
3660   2021-06-14 19:15:14
3661   2021-02-16 11:55:26
Name: deadline, Length: 3278, dtype: datetime64[ns]

In [39]:
test

,launched_at,deadline
1,1329076826,1333349940
2,1452796491,1455388491


In [40]:
test['deadline'] = pd.to_datetime(test['deadline'],unit='s')

/var/folders/fy/16bvjs25629769nm3_pgm3cw0000gn/T/ipykernel_4667/531838772.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test['deadline'] = pd.to_datetime(test['deadline'],unit='s')


In [22]:
from datetime import datetime

test = datetime.fromtimestamp(test.iloc[0,0])

In [23]:
test

datetime.datetime(2012, 2, 12, 15, 0, 26)

In [30]:
test['deadline'] = pd.to_datetime(test['deadline'],unit='s')

/var/folders/fy/16bvjs25629769nm3_pgm3cw0000gn/T/ipykernel_4667/531838772.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test['deadline'] = pd.to_datetime(test['deadline'],unit='s')
